In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np


# Load the CSV files
amplitude_spectrum = pd.read_csv('AMP_delta_theta_alpha_beta_LAHc9.csv', header=None)

binary_labels = pd.read_csv('r_r_binary.csv')  # Ensure the binary labels match the samples

# Standardize each dataset
scaler = StandardScaler()

amplitude_spectrum_standardized = scaler.fit_transform(amplitude_spectrum)


# Convert binary labels to numpy array
binary_labels = binary_labels['Binary Label'].values  

X_train_full, X_test, y_train_full, y_test = train_test_split(
    amplitude_spectrum_standardized, binary_labels, test_size=0.2, random_state=42
)

2024-11-30 18:38:07.238348: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-30 18:38:07.244755: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-30 18:38:07.265775: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733009887.299690   11976 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733009887.309517   11976 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-30 18:38:07.344860: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [ ]:
len(binary_labels)

In [2]:
from sklearn.model_selection import train_test_split

# Reserve a separate test set
X_train_full, X_test, y_train_full, y_test = train_test_split(
    amplitude_spectrum_standardized, binary_labels, test_size=0.2, random_state=42
)

# Verify the split
print(f"Training set shape (for k-fold): {X_train_full.shape}, {y_train_full.shape}")
print(f"Test set shape: {X_test.shape}, {y_test.shape}")


Training set shape (for k-fold): (202, 4), (202,)
Test set shape: (51, 4), (51,)


In [3]:
from sklearn.model_selection import KFold
# Initialize K-Fold Cross-Validation
kf = KFold(n_splits=3, shuffle=True, random_state=42)

fold_accuracies = []
fold_losses = []

# Perform k-fold cross-validation
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_full)):
    print(f"\nFold {fold + 1}")
    
    # Split into training and validation sets for this fold
    X_train, X_val = X_train_full[train_idx], X_train_full[val_idx]
    y_train, y_val = y_train_full[train_idx], y_train_full[val_idx]
    
    # Build and compile the model
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(4, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(4, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Train the model
    history = model.fit(X_train, y_train, epochs=20, batch_size=20, validation_data=(X_val, y_val), verbose=0)
    
    # Evaluate on validation set
    val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    
    fold_losses.append(val_loss)
    fold_accuracies.append(val_accuracy)

# Calculate and display mean results from k-fold validation
mean_loss = np.mean(fold_losses)
mean_accuracy = np.mean(fold_accuracies)

print("\nK-Fold Validation Results:")
print(f"Mean Validation Loss: {mean_loss:.4f}")
print(f"Mean Validation Accuracy: {mean_accuracy:.4f}")


Fold 1


/home/ruilin/Desktop/ECE9063/assignment-1-assignment-group-51/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-11-30 18:38:13.802461: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Validation Loss: 0.7046, Validation Accuracy: 0.5147

Fold 2
Validation Loss: 0.7585, Validation Accuracy: 0.4478

Fold 3
Validation Loss: 0.7214, Validation Accuracy: 0.4925

K-Fold Validation Results:
Mean Validation Loss: 0.7282
Mean Validation Accuracy: 0.4850


In [6]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print("\nFinal Test Results:")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")



Final Test Results:
Test Loss: 0.6893
Test Accuracy: 0.4902


In [24]:
import keras_tuner as kt

def build_model(hp):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(hp.Int('units', min_value=16, max_value=64, step=16), 
                              activation=hp.Choice('activation', ['relu', 'tanh'])),
        tf.keras.layers.Dense(hp.Int('units', min_value=16, max_value=64, step=16), 
                              activation=hp.Choice('activation', ['relu', 'tanh'])),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer=hp.Choice('optimizer', ['adam', 'sgd']),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

In [25]:
# Initialize the tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=100,
    executions_per_trial=1,
    directory='my_dir',
    project_name='ffnn_tuning'
)

# Search for the best hyperparameters
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, verbose=1)

# Get the best hyperparameters and model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters: {best_hps.values}")

Reloading Tuner from my_dir/ffnn_tuning/tuner0.json
Best Hyperparameters: {'units': 16, 'activation': 'tanh', 'optimizer': 'sgd'}


In [32]:

# Build the model with the best hyperparameters
final_model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='tanh', input_shape=(X_train.shape[1],)),  # First hidden layer
    tf.keras.layers.Dense(16, activation='tanh'),  # Second hidden layer
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer
])

# Compile the model with SGD optimizer
final_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = final_model.fit(
    X_train, y_train,
    epochs=20,  
    batch_size=8,
    validation_split=0.2,
    verbose=1
)

# Evaluate the model on the test set
test_loss, test_accuracy = final_model.evaluate(X_test, y_test, verbose=0)
print(f"\nFinal Test Loss: {test_loss:.4f}")
print(f"Final Test Accuracy: {test_accuracy:.4f}")


Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.4418 - loss: 0.8099 - val_accuracy: 0.6452 - val_loss: 0.6340
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4342 - loss: 0.8385 - val_accuracy: 0.6452 - val_loss: 0.6338
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4291 - loss: 0.7824 - val_accuracy: 0.6452 - val_loss: 0.6335
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5319 - loss: 0.7204 - val_accuracy: 0.6452 - val_loss: 0.6338
Epoch 5/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4386 - loss: 0.7936 - val_accuracy: 0.6452 - val_loss: 0.6341
Epoch 6/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4215 - loss: 0.7534 - val_accuracy: 0.6129 - val_loss: 0.6355
Epoch 7/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4562 - loss: 0.7370 - val_accuracy: 0.6129 - val_loss: 0.6384
Epoch 8/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4430 - loss: 0.7550 - val_accuracy: 0.6129 - val_loss